#  9. Solidity

* Last Updated 20200518MON1200 20190605WEB1200 20190120SUN0000 20180707SAT1300 20170112
* todo: 1) library ```myLib+StringUtils``` 2) 계정에서 발생한 거래의 목록? 4) registry

## 9.1 학습내용

### 9.1.1 목표
 
* solidity 파일
* value type, reference type
* bool, int, string, address 등 데이터타입을 배우기

### 9.1.2 목차

* 9.2 [Solidity File](#9.2)
    * Pragma
    * Import
    * Contract
    * comments
* 9.3 컨트랙의 구조
    * SimpleContract.sol
* 9.4 데이터 타잎
    * 9.4.1 타잎 구분
    * 실습: DataTypeTest.sol
    * 9.4.2 [계정주소](#9.4.2)
    * 9.4.3 [저장장소](#9.4.3)
    * 9.4.4 [가시성](#9.4.3)
    * 실습: 주소 계정에서 입출금 myBank.sol
    * 9.4.5 [conversion](#9.4.5)
    * 실습: DataConversionTest.sol
    * 9.4.6 [전역 변수](#9.4.6)
    * 실습: GlobalVarsTest.sol
    * 살습: 은행송금 BankV2

## 9.2 Solidity File
<a id='9.2'></a>

또한 다른 언어에 비해 **라이브러리도 충분하지 못하여** 불편할 수도 있다. 아직 발전하고 있는 언어라서 버전업이 활발하게 되고 있고, **후방호환성**이 문제가 되기도 한다.

Solidity File은 contract 소스코드를 확장자 ```.sol```로 저장한 파일이다.
파일에는 버전 pragma, 다른 소스코드를 불러오는 import문, 도움말 comments를 포함할 수 있다.

### Pragma
```pragma```는 컴파일러 버전을 선택할 때 사용한다.
프로그램의 첫 줄에 적는다.

```python
pragma solidity <버전>;
```

### Import
import는 filename의 프로그램을 읽어들여 사용하게 된다.

```python
import "filename"
```
그 파일 내 **컨트랙, 전역변수** 등을 그대로 가져오기 때문에 주의를 해야 한다.
이로 인해 현재 파일의 전역변수와 문제를 일으킬 수 있으므로
```import ... as ...``` 구문을 사용하기도 한다.

```python
import "filename" as symbolName
```

### Contract

Contract은 블록체인 상의 프로그램이지만, 그 개념이나 구현하는 객체지향언어에서 사용하는 **클래스와 유사**하다.
따라서 클래스와 같이, Contract은 생성자, 속성, 함수로 구성한다.
물론 객체지향의 상속, 다형성을 사용할 수 있다.

### Comments
<a id='S.1.4'></a>

도움말을 적는다. C 또는 자바 언어 스타일과 동일하다.
한 줄은 ```//```라고 맨 앞에 적어주어 도움말로 만든다.
여러 줄은 해당 블록 앞 뒤에 ```/*...*/```를 적어준다.
Doxygen 스타일의 @태그를 사용하는 것도 가능하다.
```python
/**
@param p1 The first parameter
@param p2 The second parameter
@returns the returned results
*/
```

## 9.3 컨트랙 구성

### 컨트랙은 멤버변수, 함수로 구성

컨트랙은 객체지향의 클래스와 같이 구성하면 된다.
클래스가 (1) 멤버 **속성**과 (2) 멤버 **함수**로 구성하듯이 컨트랙도 마찬가지이다.
객체지향으로서 상속을 사용하여 확장을 할 수도 있다.

다음 ```SimpleChild.sol```은 단순한 컨트랙의 예제이다.
* 상속 관계의 컨트랙은 ```is```로 표현
* 컨트랙의 멤버 속성 **state variables**으로 ```int```, ```address```, ```mapping```을 선언
* ```event PrintLog``` 이벤트를 선언.
* 함수 function ```constructor```, ```deposit()```, ```queryBalance()``` 또는 ```modifier```를 가질 수 있다.
* Child에서 부모의 함수를 상속받아 ```add()```, ```getCounter()```를 사용할 수 있다.

In [1]:
%%writefile src/SimpleChild.sol
pragma solidity ^0.6.0;

contract Parent {
    //state variables
    address payable owner;
    uint private counter;
    
    //constructor
    constructor() public {
        owner=msg.sender;
        counter = 0;
    }
    //functions
    function add() public { counter++; }
    function getCounter() public view returns(uint) { return counter; }
}

contract SimpleChild is Parent {
    //state variables
    string nickName;
    mapping(address => uint) private balances;
    //event
    event PrintLog(address, uint);
    
    //constructor
    constructor() public {}  
    //functions
    function setNickName(string memory s) public { nickName = s; }
    function getNickName() public view returns(string memory) { return nickName; }
    function deposit() public payable {
        balances[msg.sender] += msg.value;
        emit PrintLog(msg.sender, msg.value);
    }
    function queryBalance() public view returns (uint) {
        return balances[msg.sender];
    }
    //access non-private members of the parent
    function kill() public {
        if (msg.sender == owner) selfdestruct(owner);
    }
}

Writing src/SimpleChild.sol


### 스타일

* import문은 프로그램 위에 적는다.
* 컨트랙은 새로운 줄에 적어준다.
* 컨트랙 간에는 2줄 띄어쓰기를 해준다.
* 들여쓰기를 탭을 사용하지 않고 공백 4칸을 넣어준다.
* 배열은 ```int[] x;```이라고 적어준다 (```int [] xㅣ``` 또는 ```int x[];```가 아니라)
* 문자열은 쌍따옴표를 해준다.
* 함수는 새로운 줄에 적어준다
* 한 줄은 최대 79 문자를 넘지 않게 한다.
* 괄호에서는 한 칸 띄어쓰기를 하지 않는다. ```if (x == 1)``` (```if ( x == 1 )```이 아니라)
* 연산자 앞 뒤 1칸을 넣어준다.
* 블록 스타일은 바로 이어서 중괄호를 연결한다.

```python
if (x==1) {

}
```


### naming

* 파일명은 컨트랙명과 달라도 괜찮지만 일치하도록 한다.
* 컨트랙, Event, enum, Struct: 대문자로 시작하고, camel case 스타일로 단어의 첫글자는 대문자로 적어준다
* 함수, 함수인자, 변수, modifier: 소문자로 시작하고 camel case로 적어준다.
* 상수는 모두 대문자, underscore (DATE_OF_BIRTH)로 연결해서 적어준다.


### reserved words

Solidity에서 제공하는 예약 명령어를 알파벳 순으로 나열해보자.

"abstract", "after", "alias", "apply", "auto", "case", "catch", "copyof", "default", "define", "final", "immutable", "implements", "in", "inline", "let", "macro", "match", "mutable", "null",
"of", "override", "partial", "promise", "reference", "relocatable", "sealed", "sizeof", "static", "supports", "switch", "try", "type", "typedef", "typeof", "unchecked"

## 9.4 데이터 타잎
<a id='9.3'></a>

Solidity 언어는 블록체인에서 실행이 되는 까닭에 그 특성상 저장공간을 많이 사용할수록 비용이 발생하기 때문에 **저장공간을 효율적**으로 사용해야 한다.
다른 언어에서 지원하는 데이터타입을 큰 차이 없이 사용할 수 있지만 **데이터타입**이 자세하게 나누어져서 불필요한 낭비를 줄이고 있고, **배열을 검색**하거나 **반복문**을 사용할 때도 유의해야 한다.
그리고 소수점은 아직 지원되지 않아 float, double과 같은 자료형이 없다.

### 9.4.1 타잎 구분

데이터 타잎은 (1) value type, (2) reference type으로 크게 구분할 수 있다.

#### **Value Type**

데이터 그 자체의 **값**으로 저장되는 타입이다.
다른 변수에 할당되거나 함수인자로 전달될 경우, 값의 복사본이 되어 전달되어서 쓰이는 **pass by value**의 특성을 가진다.
```bool```, ```int/uint```, ```address```, ```bytes (1~32 바이트)```, ```enum```이 해당된다.
```int```, ```bytes (1~32 바이트)```는 최대 **32바이트**로 그 값을 저장할 수 있다.

#### **참조 타잎 Reference Type**

**32바이트를 초과할 수 있는 크기의 데이터**를 말하며, 그 저장주소를 통해 사용하는 타입이다.
변수에 할당되거나 매개변수로 전달될 경우, 그 저장주소가 복사되어 쓰이기 때문에 원본 데이터가 수정되면 따라서 변경된다.
```bytes```, ```array```, ```string```, ```Struct```, ```mapping```이 해당된다.

#### 타입의 구분 및 간단한 설명

데이터타잎 | 설명
----------|----------
bool | uint8, 0 and 1의 값을 가진다
uint<M> | unsigned integer, M은 8비트 단위로 256비트까지 가능하다 0 < M <= 256, M % 8 == 0
int<M> | signed integer, M은 8비트 단위로 256비트까지 가능하다 0 < M <= 256, M % 8 == 0
address | 주소. 크기가 20바이트이므로 **uint160** 이다.
uint, int | 숫자가 붙지 않은 경우의 타입으로 uint256, int256를 의미
bytes<M> | 바이너리 타잎, M은 32바이트까지 가능하다 0 < M <= 32. **```bytes``` (크기가 없는)는 value type이 아니다**.
string | UTF-8 문자열, **value type이 아니다**.

### literals

변수는 명칭이고, literals는 값이다.
* 정수는 소수점 없는 양수, 음수이고 1, 10, -1, -10 등을 예를 들 수 있다.
* string은 따옴표, 하나이든 쌍따옴표이든 상관없이 "jsl", 'hello' 등이 예가 될 수 있다.
* 주소는 객체로서 20바이트 16진수로 0x로 시작하는 20바이트, 40자리수이다.
* byte는 따옴표 없이 0x로 시작하는 16진수이다.

```python
uint age = 0x14;   //20 십진수
bytes2 b2 = 0xFFBB;
```

* 소수점은 아직 구현되지 않아 literals는 해당되지 않는다.

#### boolean
Boolean은 true, false 둘 중의 하나 값을 가지는 데이터타압이다.
비교연산자 (==, !=, !, >, <), 논리연산자 (&& ||)의 결과가 true, false 와 같이 boolean이 된다.

```python
bool isMarried=true;
```

#### int
```int```는 정수를 가지는 타입이고 ```signed```, ```unsigned```로 구분할 수 있다.
* ```signed int```는 양수, 음수 모두 허용되며
* ```unsigned int```는 양수만 사용할 수 있다.

그 크기는 8, 16, 24, 32 이런 식으로 8바이트 단위로 증가하고 **최대 32**바이트까지 사용할 수 있다.
크기를 적지 않으면 256비트로 간주한다. **기본 값 default value은 0**이다.
```+, -, *, /, %, **, ++, --, +=, -=```와 같은 산술연산자를 사용할 수 있다.

```python
int256 x=1;
```

#### fixed/unfixed

fixed, unfixed는 각 각 signed, unsigned 소수점 숫자를 말한다.
선언은 할 수 있지만, 값을 할당하면 ```UnimplementedFeatureError``` 오류가 발생한다.

줄 | 설명
-----|-----
1 | 컴파일러 0.6의 최신 버전으로 설정. 다음 명령문 ```^0.6.0```은 메이저버전 6, 마이너버전 0을 의미한다. 맨 앞 ```^``` caret는 메이저버전으로 시작하는 최신버전을 선택한다는 의미이다. 따라서 현재 발표된 최신 버전이 0.5.3이므로, 이를 선택하게 된다.
10 | ```update()``` 함수에서 멤버변수의 값을 갱신할 수 있다.
13 | 매개변수 ```int```를 넘겨준다. ```int```와 ```uint```의 형변환이 필요하다.
23 | ```assert()```가 오류이면 ```Exception```이 발생하고, 다음 줄이 실행이 되지 않는다. 즉 ```x==1```인 경우 ```true```가 반환된다.

In [4]:
%%writefile src/IntBool.sol
pragma solidity ^0.6.0;

contract IntBoolTest {
    bool married = true;
    uint256 xAge = 22;
    uint256 yAge = 25;
    //fixed phi; // = 3.14;
    function update() public {
        xAge = yAge;
        yAge = 33;
    }
    function setXAge(int age) public {
        xAge = uint(age);  //converstion
    }
    function getXAge() public view returns(uint) {
        return xAge;
    }
    function getYAge() public view returns(uint) {
        return yAge;
    }
    function testInt() public view returns(bool) {
        assert(xAge>=20 && yAge>=20);
        return true;
    }
    function isMarried() public view returns(bool) {
        return married;
    }
}

Overwriting src/IntBool.sol


#### 고정크기 byte array

```bytes```는 ```bytes1``` ~ ```bytes32``` 사이의 어떤 크기로 선언할 수 있다.
그러니까 **최대 32바이트까지 1바이트 단위**로 크기를 늘릴 수 있다.
byte, bytes32는 hex 값 뿐만 아니라 정수, 문자, 문자열을 가질 수 있다.
```byte```는 1 바이트, 8비트의 길이를 가진다. 따라서 ```byte``` (bytes가 아님)는 ```bytes1```과 동일하다. 

**bytes literals는 따옴표 없이 0x를 붙여서** 0xFF와 같다.
문자열을 입력할 수 있지만, hex 값이 저장된다.

```python
bytes1 x= 0xFF;
bytes23 place1 = "7 hongji-dong jongro-gu";
bytes8 place2 = "7 hongji"; //결과값은 0x3720686f6e676a69
```

문자 | ascii | hex
-----|-----|-----
7 | 55 | 37
space | 32 | 20
h | 104 | 68
o | 111 | 6f
n | 110 | 6e
g | 103 | 67
j | 106 | 6a
i | 105 | 69

#### 변동크기 byte array

```bytes```는 고정크기의 그것과 혼돈하기 쉬운데, 여기서는 크기를 뜻하는 **숫자가 없는** 경우를 말한다. 또는 ```string```은 그 크기가 정해져 있지 않아서 가변적이다.
길이 제한이 없는 경우 사용한다.
string은 UTF8 형식으로 문자열을 저장한다.

크기를 알려주는 ```bytes.length```를 사용할 수 있다.
```string```의 ```length```는 ```bytes(string).length```로 **형변환을 ```bytes```로 한 후**에야만 가능하다.
배열의 인덱스를 넣어 ```bytes(s)[2]```로 3번째 값을 읽을 수 있다.

```python
string s = "hello";
```

상수는 ```constant```로 표현한다.
```python
string constant name="jsl";  //상수는 ```constant```로 표현한다.
```


줄 | 설명
-----|-----
1 | 컴파일러 0.6의 최신 버전으로 설정
5 | ```bytes1```은 ```byte```과 동일한 의미
6 | ```bytes23```에는 23글자를 입력. 한 글자를 추가하면 오류가 발생된다.
7 | ```bytes8```에는 8글자를 입력.
9 | ```myBytes```의 기본 값은 0, 3바이트 이므로 0x000000
10 | ```string```은 utf-8로 저장되므로, 16진수가 아니라 "jsl"
12 | ```byte1```을 ```byte```로 반환하는 경우 형변환이 필요없다.
17 | ```bytes1```로 선언된 b1을 ```byte```로 반환한다. ```byte```와 ```bytes1```은 동일한 의미.
18 | ```bytes23```과 같이 정해진 경우, memory를 사용하지 않는다. 그러나 bytes와 같은 ref type은 memory를 사용한다.
22 | ```bytes```는 동적배열이다. ```getBytes()```는 3글자로 설정된 동적배열을 memory로 반환한다. 그 반환 값이 16진수로 출력.
24~27 | ```bytes23.length```는 23이 반환된다.
27~31 | 가변크기 ```bytes.length```는 23이 반환된다. 같은 값을 가지고 있는 ```place23```과 같은 크기.
34-36 | 매개변수는 hex로 넣어준다. a의 hex는 61이다. bytes1은 "0x61",  bytes2 "0x6161"로 입력한다. 2바이트 넘거나, utf-8 문자열을 넣으면 오류이다.
37-39 | ```myBytes```는 3바이트로 할당되었으므로 크기에 맞추어 넣어준다. UTF-8로 넣어주어도 되고 **"smu"로 넣어주면 0x736d75**로 hex값으로 저장된다
40~44 | <string>.length는 오류. 반드시 ```string```을 ```bytes```로 변환해서 ```length```를 구한다.
46~49 | 한글을 입력하려면 string을 사용한다.

In [3]:
%%writefile src/ByteStringTest.sol
pragma solidity ^0.6.0;
contract ByteStringTest {
    byte b = 0xFF;
    bytes1 b1 = 0xFF;
    bytes2 b2 = 0xFFAA;
    bytes8 place8 = "7 hongji";
    bytes23 place23 = "7 hongji-dong jongro-gu";
    bytes place = "7 hongji-dong jongro-gu Seoul"; //variable length
    bytes myBytes = new bytes(3);  //0x000000
    string constant name = "jsl"; //utf-8 string "jsl"
    function getB1() public view returns(byte) {
        return b1;  //byte, so no casting required
    }
    function getB2() public view returns(bytes2) {
        return b2;
    }
    function getB23() public view returns(bytes23) {
        return place23;  //fixed size, value type (no memory)
    }
    /**@return hex bytes. reference type should be set as memory*/
    function getBytes() public view returns(bytes memory) {
        return myBytes;  //smu in hex 0x736d75
    }
    function getLengOfBytes23 () view public returns(uint) {
        return place23.length;  // returns 23
    }
    function getLenOfBytes() pure public returns(uint) {
        bytes memory bm = "7 hongji-dong jongro-gu";
        return bm.length;        // returns 23
    }
    //need the arg in hex e.g. bytes1 0x61 bytes2 0x6161
    //a 61, b 62, ... , y 79
    //try invalid type, e.g. bytes2 0x61 or 0x616161
    function setB2(bytes2 _b2) public {
        b2=_b2;
    }
    function setBytes() public {
        myBytes="smu";
    }
    function getLenOfString() pure public returns(uint) {
        string memory nameLocal="jslLocal";
        //return nameLocal.length;  //error, casting required
        return bytes(nameLocal).length;
    }
    function getString() pure public returns(string memory) {
        string memory s = "한글";
        //bytes memory b4 = "한글";  //ok. bytes is a ref type, so memory used
        return s;
    }
}

Overwriting src/ByteStringTest.sol


#### struct

struct은 서로 관련있는 데이터를 그에 맞는 데이터타잎으로 구성하여 묶어서 사용할 수 있다.
CapWords 스타일로 첫글자는 대문자로 적어준다.
Student를 uint, string, bool을 가지게 만들면 다음과 같다.

```python
struct Student {
        uint num;
        string name;
        bool isEnrolled; 
}
Student s1=Student(201911111,"jslim",true); 
```

줄 | 설명
-----|-----
1 | 컴파일러 0.6의 최신 버전으로 설정
12 | ```uint```, ```bool```은 value 타입이고 ```string```은 reference 타입이라 ```memory``` 공간을 사용한다.
17~22 | ```struct```은 반환할 수 없다. 항목 하나 하나를 반납한다. **web3에서 반환할 경우에는 문자열로 변환하여서 toString() 출력**할 수 있다.
24~26 | 새로운 ```Student```를 지역변수 ```s3```로 만들고, 이름만 반환한다. 주의할 점은 s3를 ```storage```, ```memory``` 선택할 수 있으나 오른쪽에서 ```Student```를 지역에서 생성하고 있으므로 ```memory```를 사용하도록 적는다.

In [2]:
%%writefile src/StructTest.sol
pragma solidity ^0.6.0;
contract StructTest {
    struct Student {
        uint num;
        string name;
        bool isEnrolled;
    }
    Student s1=Student(201911111,"jslim",true);
    Student s2;
    //memory only for string type
    //201711111,"kim",false
    function setStudent2(uint n, string memory sn, bool e) public {
        s2.num = n;
        s2.name = sn;
        s2.isEnrolled = e;
    }
   function getStudent1() public view returns(uint, string memory, bool){
       return (s1.num, s1.name, s1.isEnrolled);
   }
   function getStudent2() public view returns(uint, string memory, bool){
       return (s2.num, s2.name, s2.isEnrolled);
   }
   function getStudentName() pure public returns(string memory) {
       //the right is locally created, so memory (not storage) is declared
       Student memory s3 = Student(201911112, "jsl3", true);
       return s3.name;
   }
}

Overwriting src/StructTest.sol


#### enumeration

```enum```은 요소로 구성된 데이터 타잎을 정의할 경우 사용한다.
예를 들면, 요일과 같이 월, 화, 수, 목, 금, 토를 '집합'을 구성할 경우 ```enum```을 사용한다. 
요소는 **0부터 시작하여 정수 값**을 가진다.
**비교**에 사용하며, 존재하지 않는 선택으로 인한 오류를 막을 수 있어 유용하다.
아래는 성별을 남, 녀로 구성한 ```enum```이다. 앞 글자는 대문자로 적어준다.

```python
enum Gender {male, female}
```

요일을 ```enum```으로 구성하면 다음과 같다.
```python
enum Day {SUNDAY, MONDAY, TUESDAY, WEDNESDAY, THURSDAY, FRIDAY, SATURDAY}
```


줄 | 설명
-----|-----
1 | 컴파일러 0.6의 최신 버전으로 설정
4 | **```enum```이름.변수명**으로 사용하고, 요소는 index로 읽거나 출력한다.
8 | ```getMyDay()```는 정수 값을 반환한다.
11 | 매개변수를 넘겨줄 때, ```Day``` 타입을 맞추어 줄 수 없다면 정수를 적어준다.
17 | 요소에 해당하는 값을 index로 설정할 수 있다.

In [1]:
%%writefile src/EnumTest.sol
pragma solidity ^0.6.0;
contract EnumTest {
    enum Day {MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY,SUNDAY}
    Day myDay = Day.FRIDAY; //index int4
    
    /* @return Day  returning index*/
    function getMyDay() public view returns(Day) {
        return myDay;   //index
    }
    /* @param d  ok to pass an integer (uint8)
    function setMyDay(Day d) public {
        myDay = d;
    }
    //uint is converted to uint8, which is default
    function setMyDayInt(uint d) public {
        myDay = Day(d);
    }
}

Overwriting src/EnumTest.sol


#### 배열

1) 고정배열은 그 크기가 사전에 정해지게 된다.
2) 반면에 동적 배열은 그 크기가 사전에 정해지지 않는다.
배열의 크기는 고정, 변동배열 모두 ```length```로 알 수 있다.
동적배열에 대해서 요소를 추가하거나 ```push```, 제거하거나 ```pop``` 함수를 사용할 수 있다.

* ```length```: 배열의 길이, 즉 몇 개의 요소가 포함되어 있는지 출력한다.
* ```push```: 배열에 요소를 추가하고, 동적배열, ```bytes```에 사용할 수 있고 ```string```에는 쓰지 못한다.
* ```pop```: 배열에서 요소를 제거하는 함수이다. 동적배열, ```bytes```에 사용할 수 있고 ```string```에는 쓰지 못한다.

In [6]:
%%writefile src/ArrayTest.sol
pragma solidity ^0.6.0;
contract ArrayTest {
    uint[3] ages = [15, 25, 35];
    int[] marks; //dynamic ArrayTest
    
    /* @param index  array index
       @param val    value at the index*/
    function updateAges(uint index, uint val) public {
        if(index>=0 && index <=2)
            ages[index] = val;
    }
    function initMarks() public {
        marks = new int[](5);   // default 0
    }
    function appendMark(int mark) public {
        marks.push(mark);
    }
    function popMark() public {
        marks.pop();
    }
    /* @return dynamic array, so memory is used*/
    function getMarks() public view returns(int[] memory) {
        return marks;
    }
    /* @return fixed array, so memory is used*/
    function getAges() public view returns(uint[3] memory) {
        return ages;
    }
    function getLenOfArr() pure public returns(uint) {
        //memory is used because locally created array is assigned
        uint8[3] memory intArr = [0, 1, 2];
        return intArr.length;
    }
}

Overwriting src/ArrayTest.sol


### 9.4.2 address
<a id='9.4.2'></a>

address는 계정의 주소로서 **20 바이트** 길이를 가진다.
**address 객체**로서 잔고를 조회하거나, 입출금을 할 수 있으며, 이 경우 ```address payable```로 선언해야 한다.

### 잔고조회

계정의 잔액 wei를 조회할 수 있고, 반드시 자신이 아니더라도 누구나 읽을 수 있다.
현재 컨트랙의 잔고는 ```address(this).balance```로 구할 수 있다.
```this.balance```와 같이 하면 ```this```는 현재 인스턴스를 의미하기 때문에 잔고를 구할 수 없다.
Solidity 0.5.0부터 ```this.balance```는 사용이 금지되었고, ```address(this).balance```를 사용해야 한다.
this는 contract 자신의 주소를 말한다.

```python
this.myFunction()
```

#### 송금

또한 **출금**을 할 수 있다.
은행거래에서도 타인의 계좌에서 임의로 인출이 허용되지 않는 것처럼
**출금은 자신의 주소에서만 가능**하다.
자신의 주소는 스스로 unlock, sign을 할 수 있는 권한이 있고, 그에 따라 출금이 가능한 것이 당연한다.
그러나 임의의 주소에서 누군가 출금을 한다는 것은 비상식적이다.
금액의 단위는 Wei이다.
물론 **입금은 어느 계정으로든 가능**하다.

0.5.0 버전부터 주소에서 transfer() 함수를 사용하여 송금하려면 
```address payable```로 선언해야 한다.

```python
address payable owner;
```

#### send()

send() 함수는 계정으로 입금을 하고, 그 결과를 **성공 또는 실패를 나타내는 boolean으로 반환**한다.
문제는 출금계정에서 출금은 했으나, 수신계정으로 입금이 실패했을 경우에 발생한다.
송금이 성공적으로 이루어지 않았다면, ```call statck error``` 또는 ```out of gas error``` 등의 이유로 실패하는 경우에도 **예외 **Exception이 발생하지 않고 false를 반환**하게 된다.

송금이 실패한 경우에 주의해야 하며, 금액을 출금계정으로 반환하거나, 적절한 처리 로직을 넣어 주어야 한다.
Solidity 0.4.10 이전에는 ```if(!address.send(amount)) throw;```와 같이 코딩을 했으나,
```require(address.send(amount)```로 코딩하도록 한다.

receiver.send()에 대해, receiver fallback함수가 실행되면, 최대 2300 gas를 사용할 수 있다. 2300을 초과하는 금액저장, 송금, 다른 컨트랙을 호출하면 out-of-gas 예외가 발생한다.
    
```throw```는 송금액이 원위치, 송금자에게 반환된다. gas는 당연히 소비되었으므로 송금반환되지 않는다. gas 잔액이 남아도 반환되지 않고 모두 소비된 것으로 한다. 여러 계정에 송금하는 중 throw가 발생하면, 나머지 송금계정에는 송금이 아예 발생하지 않을 수 있다.

```python
address _receiver = 0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480;
if(myAddress.balance>=10) _receiver.send(10);
```

#### transfer()

실패하면 throw 예외를 발생시키며, 호출자에게 환급된다. 따라서 send() 보다 안전하다.

```python
address.transfer(amount)
```

#### call (두 계약 사이에 call하는 경우는 다음에 설명)

call, callcode, delegatecall은 low-level이므로 가급적 사용하지 않는다.
call함수는 gas를 정해서 처리하는 경우에 사용한다.
이 함수는 addr.send(x)와 동일한 기능을 할 수 있다. 단, 받는 측에 gas를 사용할 수 있게 하기 때문에 악의적으로 사용될 수 있다.

```python
<address>.call.value(20) //20 Wei를 송금한다.
```

**받는 측 함수를 실행하면서 송금**.
함수를 호출, gas는 2, 송금액은 20 Wei. 이 경우 gas금액이 적어서 실패하게 될 것이다.
```python
<address>.gas(2).value(20)()
```

```deposit```을 호출하려면 ```value```에 ether를 넣어주어야 한다.
송금하려면 ```sendTo()```에 주소를 넣어 주어야 하는데, address literals은 따옴표 없이 넣어준다. 따옴표를 하면 문자열 타입으로 인식하게 된다.

![alt text](figures/9_depositSendValue.png "put amount in value and address without quotes")

In [2]:
%%writefile src/AddressTest.sol
pragma solidity ^0.6.0;
contract AddressTest {
    address owner;
    address payable receiver;
    uint balanceOfOwner;
    constructor() public {
        owner=msg.sender;
        //myBalance = msg.sender.balance;
        balanceOfOwner = owner.balance;
    }
    function deposit() payable public {
    }
    /* @param addr  set as payable because it will get some gwei*/
    function setReceiver(address payable addr) public {
        receiver=addr;
    }
    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;  //balance of contract
    }
    function getBalanceOfOwner() public view returns(uint) {
        return owner.balance;
    }
    function getBalanceOfReceiver() public view returns(uint) {
        return receiver.balance;
    }
    function sendTo() public payable {
        require(receiver.send(111)); //send 111 gwei to xAddress
        //msg.sender.transfer(address(this).balance);
    }
}

Writing src/AddressTest.sol


### 9.4.3 저장장소
<a id='9.4.3'></a>

reference type은  ```Arrays```, ```struct```, ```mapping```과 같이 32바이트 이상의 크기를 가진 복잡한 데이터 타잎이다. 이들 변수는 저장장소를 정의해 줄 수 있다.

* ```storage```는 32바이트 길이의 key-value 형식으로 저장되고 계속 유지되는 경우이다. 컨트랙의 모든 함수에서 값을 볼 수 있다. 컨트랙의 상태변수는 기본 값이 ```storage```이며 일단 저장이 되면 영원히 저장된다.
* ```memory```는 단기간 저장하는 경우에 사용된다. **지역변수**는 함수 내에서만 짧은 시간 동안 사용되므로 memory 수식어를 붙여준다. **함수의 인자 또는 반환 값은 memory**를 사용한다. 단 **struct, array or mapping과 같은 reference type은 지역변수라 하더라도 storage**가 기본 값이다.
* ```stack```: 지역변수 (array, struct, mapping 제외)를 저장하는 stack. 1,024 수준까지 저장할 수 있고 그 이상은 예외처리 된다. 비용은 memory와 동일하다.
* ```calldata```: **외부**에서 함수를 호출할 때 호출자가 제공하는 함수인자를 저장한다.

요약하면,
* 상태변수는 storage에 저장한다.
* 지역변수 (함수인자/반환, value type 지역변수)는 memory를 기본 default로 사용한다.
단, 함수에서 사용되는 reference type (struct, array, mapping, string)은 기본이 storage이고, memory를 사용하려면 선언이 필요하다.

### 9.4.4 가시성
<a id='9.4.4'></a>

지역변수는 자신이 선언된 함수에 국한되어 사용할 수 있다.
반면에 멤버변수 state varaiable은 사용할 수 있는 범위를 제어할 수 있다.

* 가시성을 적어주지 않으면 기본 값은 ```internal```로 정의된다.
```internal```은 컨트랙 내부에서만 사용할 수 있고, 상속을 하면 자식이 물려받을 수 있다는 의미이다.
* ```external```은 ```internal```의 반대, 외부에서 호출하는 경우만 허용된다. 내부에서 사용할 경우 ```this.f()```로 호출함. 
* 외부에서 사용하려면 ```public```으로 선언하면 된다. 객체지향 ```public```과 동일한 의미이다.
* 블록체인 내부에서 컨트랙 자신만 사용할 수 있고, 객체지향 ```private```과 동일한 의미이다.

구분 | 설명 | default
-----|-----|-----
```public``` | 블럭체인 외부에서 누구나 사용 | no
```private``` |  컨트랙 자신만 사용 | no
```internal``` | 현재 컨트랙 내부에서만 또는 상속의 경우에 사용하는 경우. java의 ```protected```와 같은 의미 | yes
```external``` | 외부에서 호출하는 경우만 허용 | no


### 연산자

* 산술연산자: +  - ```*```  /  ```%``` (modulus) ++ -- ```**``` (exponentiation)
* 비교연산자: ```==``` ```!=``` ```>``` ```<``` ```>=``` ```<=```
* 논리연산자: && || ! (부정)
* 비트연산자: & | ^ ~
* 할당연산자: ```=``` ```+=``` ```-=``` ```*=``` ```/=``` ```%=``` (```x = x % y```는 ```x %= y```)
* 삼항연산자: ```?:```

## 실습: 주소 계정에서 입출금

앞서 geth 단말에서 account[0]에서 account[1]로 전송해보았다.
컨트랙에서 송금하는 프로그램을 개발해 보자.
은행을 경유하지 않고도 송금이 이루어지는 과정을 이해해 보자.
은행없이 송금하는 경우, 서로 어떻게 신뢰할 수 있는지
실패하는 경우에는 어떻게 복구할 수 있는지 생각해 보자.


### 단계 1: 컨트랙 개발

줄 | 설명
-----|-----
5 ~ 8 | 전송자의 ```msg.value```에 입력된 ether를 contract으로 입금하는 함수이다. 주의할 점은 함수인자 amount는 ether가 아니다. 실제 입금되는 금액은 ```msg.value```이다. 단 ```msg.value```와 amount가 동일해야 입금이 실행된다. 그리고 **```payable```**로 명시해 주어야 한다.
9 ~ 11 | ```msg.sender``` 계정으로 컨트랙 자신의 잔고, ```address(this).balance```를 전부 이체한다. ```msg.sender```는 매번 메시지를 전송할 때마다 변경되는 것이 아니라, 배포하는 시점의 전송자 즉 프로그램을 배포하는 권한을 가진 사람만이 (정확히 말하면 개인키) owner가 된다.
12 ~ 14 | 컨트랙 자신의 잔고, ```address(this).balance```를 조회한다.

In [5]:
%%writefile src/myBank.sol
pragma solidity ^0.5.0;

contract myBank {
    function deposit(uint256 amount) payable public {
        //'value' of transacion object
        require(msg.value == amount);
    }
    function withdraw() public {
        msg.sender.transfer(address(this).balance);
    }
    function queryBalance() public view returns (uint256) {
        return address(this).balance;
    }
}

Overwriting src/myBank.sol


### 단계 2: 컴파일

In [6]:
!solc --gas --abi --bin src/myBank.sol


======= src/myBank.sol:myBank =======
Gas estimation:
construction:
   129 + 77200 = 77329
external:
   deposit(uint256):	259
   queryBalance():	601
   withdraw():	infinite
Binary: 
608060405234801561001057600080fd5b50610182806100206000396000f3fe608060405260043610610051576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100565780633ccfd60b14610081578063b6b55f2514610098575b600080fd5b34801561006257600080fd5b5061006b6100c6565b6040518082815260200191505060405180910390f35b34801561008d57600080fd5b506100966100e5565b005b6100c4600480360360208110156100ae57600080fd5b8101908080359060200190929190505050610145565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610142573d6000803e3d6000fd5b50565b803414151561015357600080fd5b5056fea165627a7a723058200be23183b595a23b19c1edd28a441bd6edb3c079b72867b70

### 단계 3: 컨트랙 배포

In [8]:
%%writefile src/myBankDeploy__.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50610182806100206000396000f3fe608060405260043610610051576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100565780633ccfd60b14610081578063b6b55f2514610098575b600080fd5b34801561006257600080fd5b5061006b6100c6565b6040518082815260200191505060405180910390f35b34801561008d57600080fd5b506100966100e5565b005b6100c4600480360360208110156100ae57600080fd5b8101908080359060200190929190505050610145565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610142573d6000803e3d6000fd5b50565b803414151561015357600080fd5b5056fea165627a7a723058200be23183b595a23b19c1edd28a441bd6edb3c079b72867b70c48245fbfc5bddb0029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new("hello world",{data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Writing src/myBankDeploy.js


In [2]:
%%writefile src/myBankDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));  //ganache-cli
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50610182806100206000396000f3fe608060405260043610610051576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100565780633ccfd60b14610081578063b6b55f2514610098575b600080fd5b34801561006257600080fd5b5061006b6100c6565b6040518082815260200191505060405180910390f35b34801561008d57600080fd5b506100966100e5565b005b6100c4600480360360208110156100ae57600080fd5b8101908080359060200190929190505050610145565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610142573d6000803e3d6000fd5b50565b803414151561015357600080fd5b5056fea165627a7a723058200be23183b595a23b19c1edd28a441bd6edb3c079b72867b70c48245fbfc5bddb0029";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from:"0xf84B27A6D281Df793C67Dc12A2D69FF003BB5732",gas:1000000}, function(err, transactionHash) {
        if (!err) {
            console.log("transactionHash: ", transactionHash);
        }
    })
    .then(console.log);

Overwriting src/myBankDeploy.js


In [3]:
!node src/myBankDeploy.js

transactionHash:  0xc37176b6c3a287db11e930b485e5bf817431e8cf7256e5adf546731ed088ad12
Contract {
  currentProvider: [Getter/Setter],
  _requestManager: RequestManager {
    provider: HttpProvider {
      withCredentials: false,
      timeout: 0,
      headers: undefined,
      agent: undefined,
      connected: true,
      host: 'http://127.0.0.1:8345',
      httpAgent: [Agent]
    },
    providers: {
      WebsocketProvider: [Function: WebsocketProvider],
      HttpProvider: [Function: HttpProvider],
      IpcProvider: [Function: IpcProvider]
    },
    subscriptions: {}
  },
  givenProvider: null,
  providers: {
    WebsocketProvider: [Function: WebsocketProvider],
    HttpProvider: [Function: HttpProvider],
    IpcProvider: [Function: IpcProvider]
  },
  _provider: HttpProvider {
    withCredentials: false,
    timeout: 0,
    headers: undefined,
    agent: undefined,
    connected: true,
    host: 'http://127.0.0.1:8345',
    httpAgent: Agent {
      _events: [Object: null prototype

### 단계 4: 사용

```deposit()```함수의 ```value:1111``` 필드를 채워주면 ```msg.value```로 전달이 된다.
이 때 함수의 인자도 동일하게 1111 Wei를 넣어준다.

```python
myBank.deposit(1111,{from:web3.eth.accounts[0],gas:80000,value:1111})
```

마이닝을 하고나면 잔고가 1111이 된다.
컨트랙의 ```deposit()``` 함수에서 잔고에 합산을 하지 않아도 된다.

자신의 잔고에서 1111만큼 빠져나간 것을 확인해보자.
```python
> eth.getBalance(eth.accounts[0]);
94999999999999998889
```

In [9]:
%%writefile src/myBankUse__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"}];
var _contract = web3.eth.contract(_abiArray);
var myBank = _contract.at("0x646570db59e036e51a881f5e46722216ccbe7f52");
console.log(web3.eth.getBalance(web3.eth.accounts[0]).toNumber());
console.log(myBank.deposit(1111,{from:web3.eth.accounts[0],gas:80000,value:1111}));
var bal=myBank.queryBalance.call();
console.log(bal.toString());
console.log(web3.eth.getBalance(web3.eth.accounts[0]).toNumber());
myBank.withdraw.sendTransaction({from:web3.eth.accounts[0]});

Overwriting src/myBankUse.js


In [6]:
%%writefile src/myBankUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"}];
var myBank = new web3.eth.Contract(_abiArray, '0x657f2cc19b9A634e62d88E419a73DAf0AB738397');
myBank.methods.queryBalance().call().then(console.log);
myBank.methods.deposit(1111).send({from:"0xf84B27A6D281Df793C67Dc12A2D69FF003BB5732",gas:80000,value:1111});
myBank.methods.queryBalance().call().then(console.log);
//myBank.methods.withdraw().send({from:"0xf84B27A6D281Df793C67Dc12A2D69FF003BB5732"});
//myBank.methods.queryBalance().call().then(console.log);

Overwriting src/myBankUse.js


In [8]:
!node src/myBankUse.js

2222
2222


1111을 deposit() 입금하면 잔고에 반영이 되는 것을 볼 수 있다.
또한 withdraw() 출금하면 잔고가 0이 되면서 전송자의 잔고가 증가한다.

In [11]:
!node src/myBankUse.js

760000000000000000000
0x4026b7152afdffc69b56c5799e7bce73ce886687a7f0a4e313018c84a772a1d3
0
760000000000000000000


### 9.4.5 conversion
<a id='9.4.5'></a>

형변환을 할 경우에는 데이터 타잎의 크기에 주의해야 한다.
예를 들어 int8에서 int로 변환할 경우 괄호 안에 써주면 된다.
이 경우 int는 256비트이므로 오류없이 변환이 된다.
```python
int8 i8=20;
uint i256=int(i8);
```

uint는 bytes32로 변환할 수 있다.
```python
uint x=20;
bytes32(x);
```

int를 bytes로 바로 형변환을 할 수 없다.
int는 int256, 즉 32바이트라서 bytes32로 변환하고, 이를 bytes로 변환해야 한다.
이 경우 바이트마다 옮겨주어야 한다.

```0x68656c6c```은 16진수, 8자리이다.
한 자리에는 0,1,2...D,E,F까지 $2^4$ 가지를 표현할 수 있고, 4비트가 소요된다.
이 데이터 bytes4를 bytes2로 변환해보자.
**오버플로우는 위 4자리부터 잘려 나가게 된다**.
따라서 ```0x6865```가 저장된다.

```python
bytes4 b4 = 0x68656c6c;
bytes2(b4);
```

```string```을 ```bytes```로 변환하는 경우:

```python
string s = "hello";
bytes b = bytes(s); //0x68656c6c6f
```

hex | char
-----|-----
68 | h
65 | e
6c | l
6c | l
6f | o

In [ ]:
```address payable```에서 ```address```로의 형변환은 가능하지만, 반대는 허용되지 않는다.
```address``` -> ```uint160``` -> ```address payable```로 20바이트의 정수로 변환한 후 하도록 한다.

In [2]:
%%writefile src/DataConversionTest.sol
pragma solidity ^0.5.0;
contract DataConversionTest {
    function convertInt8ToInt() view public returns(int) {
        int8 i8=20;
        return int(i8);
    }
    function convertIntToBytes32() view public returns(bytes32) {
        uint x=20;
        return bytes32(x);
    }
    function convertInt64ToBytes8() view public returns(bytes8) {
        uint64 x=30;
        return bytes8(x);
    }
    function convertIntToBytes() view public returns(bytes memory) {
        // can not convert uint -> bytes
        // convert uint -> bytes32 -> bytes
        uint x = 20;
        bytes32 y = bytes32(x); //uint=uint256
        // can not convert from bytes32 -> bytes;
        bytes memory z = new bytes(32);
        for (uint i=0; i < 32; i++) {
            z[i] = y[i];
        }
        return z;
    }
    function convertStringToBytes() view public returns(bytes memory) {
        string memory s="hello";
        return bytes(s);  //0x68656c6c6f
    }
    function convertBytes4ToBytes2() view public returns(bytes2) {
        bytes4 b4=0x68656c6c;
        return bytes2(b4);
    }
    function convertBytes4ToInt32() view public returns(int32) {
        bytes4 b4=0x68656c6c;
        return int32(b4);
    }
}

Writing src/DataConversionTest.sol


### 9.4.6 전역 변수
<a id='9.4.6'></a>

* 화폐 단위 ```wei``` $10^{18}$로 표현되며, 1 ```ether```는 ```1000000000000000000```이다.

```python
1 ether == 1000 finney
```

* 시간 ```years```는 지원되지 않으며, ```seconds```로 표시된다. ```now```는 현재 시간, ```block.timestamp```와 동일하다.
```python
1 == 1 seconds, 1 days == 86400 seconds
```

```python
uint mytime=now;
```

* tx 컨트랙을 호출하는 트랜잭션 관련 정보
    * ```tx.origin``` 트랜잭션에 사인한 계정
    * ```tx.gasprice``` 트랜잭션 호출자가 명시한 gas price

* msg 컨트랙의 함수를 호출한 전송 관련 정보
    * ```msg.data``` | call 데이터 (bytes 값으로 표현)
    * ```msg.gas``` | gas 잔여분
    * ```msg.sender``` | 현재 함수를 호출하는 측의 주소
        * 거래가 호출자U1 -> 컨트랙C1 -> 컨트랙C2의 순서대로 완성이 될 경우:
        * C2에서는 ```msg.sender```는 바로 직전의 호출자C1를 말하며, 따라서 컨트랙도 가능하다.
        * 반면에 ```tx.origin```는 컨트랙이 될 수 없다.
    * ```msg.value``` | 컨트랙에 지급되는 ether (단위는 wei)

* block
    * ```block.coinbase``` | 현재 블록 마이너의 주소
    * ```block.difficulty``` | 현재 블록의 난이도
    * ```block.gaslimit``` | 현재 블록의 gaslimit
    * ```block.number``` | 현재 블록 수
    * ```block.blockhash``` | 현재 블록 해쉬 값
    * ```block.timestamp``` | 현재 블록 타임스탬프 epoch (1970년 1월 1일 0시) 이후 지나간 초, uint256.

In [4]:
%%writefile src/GlobalVarsTest.sol
pragma solidity ^0.5.0;
contract GlobalVarsTest {
    function getEther() view public returns(uint) {
        return 1 ether;  //1000000000000000000
    }
    function getDays() view public returns(uint,uint) {
        require(block.timestamp==now);
        return (now, 1 day); // 1558816133 86400 ??web3? 1 days?
    }
    function getMsgValue() public payable returns(uint) {
        return msg.value;
    }
    function getMsgSender() view public returns(address) {
        return msg.sender;
    }
    function getCoinbase() view public returns(address) {
        return block.coinbase;
    }
    function getBlockNumber() view public returns(uint) {
        return block.number;
    }
    function getBlockTimeStamp() view public returns(uint) {
        return block.timestamp;
    }
}

Overwriting src/GlobalVarsTest.sol


## 실습: 은행 송금

앞서 컨트랙을 수정해서 다른 계정으로 송금할 수 있도록 해보자.

### 단계 1: 컨트랙 개발

줄 | 설명
-----|-----
4 | ```address```선언
5 ~ 7 | 생성자에 ```msg.sender```를 ```owner```로 설정
8 | ```event``` 설정
10 ~ 14 | 송금. ```owner```만 송금할 수 있고 ```event``` 발생
21 ~ 24 | ```owner```에게 전액 출금

In [12]:
%%writefile src/BankV2.sol
pragma solidity ^0.5.0;

contract BankV2 {
    address owner;
    constructor() public {
        owner=msg.sender;
    }
    event Sent(address from, address to, uint amount );
    //function send(address payable _receiver, uint _amount) public payable {
    function sendTo(address payable _receiver) public payable {
        require(msg.sender==owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function queryBalance() public view returns(uint) {
        return address(this).balance;
    }
    function deposit(uint amount) public payable {
        require(msg.value==amount);
    }
    function widthdraw() public {
        require(msg.sender==owner);
        msg.sender.transfer(address(this).balance);
    }
}

Overwriting src/BankV2.sol


### 단계 2: 컴파일

In [13]:
!solc --gas --abi --bin src/BankV2.sol


======= src/BankV2.sol:BankV2 =======
Gas estimation:
construction:
   20486 + 176000 = 196486
external:
   deposit(uint256):	259
   queryBalance():	601
   sendTo(address):	infinite
   widthdraw():	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610370806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c611461006157806352b50a2a1461008c578063b6b55f25146100a3578063e6d25245146100d1575b600080fd5b34801561006d57600080fd5b50610076610115565b6040518082815260200191505060405180910390f35b34801561009857600080fd5b506100a1610134565b005b6100cf600480360360208110156100b957600080fd5b81019080803590602001909291905050506101ef565b005b610113600480360360208110156100e757600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610200565b005b60003073fffffffffffffffffffffffffff

### 단계 3: 컨트랙 배포

In [14]:
%%writefile src/BankV2Deploy__.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610370806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c611461006157806352b50a2a1461008c578063b6b55f25146100a3578063e6d25245146100d1575b600080fd5b34801561006d57600080fd5b50610076610115565b6040518082815260200191505060405180910390f35b34801561009857600080fd5b506100a1610134565b005b6100cf600480360360208110156100b957600080fd5b81019080803590602001909291905050506101ef565b005b610113600480360360208110156100e757600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610200565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561018f57600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f193505050501580156101ec573d6000803e3d6000fd5b50565b80341415156101fd57600080fd5b50565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561025b57600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156102a1573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a15056fea165627a7a723058202e512a1df8dbf7513c14646ff33b94f8e9ee902911138575219f157ffdc92ddc0029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Writing src/BankV2Deploy.js


In [3]:
%%writefile src/BankV2Deploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.WebsocketProvider('http://127.0.0.1:8345'));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610370806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c611461006157806352b50a2a1461008c578063b6b55f25146100a3578063e6d25245146100d1575b600080fd5b34801561006d57600080fd5b50610076610115565b6040518082815260200191505060405180910390f35b34801561009857600080fd5b506100a1610134565b005b6100cf600480360360208110156100b957600080fd5b81019080803590602001909291905050506101ef565b005b610113600480360360208110156100e757600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610200565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561018f57600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f193505050501580156101ec573d6000803e3d6000fd5b50565b80341415156101fd57600080fd5b50565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561025b57600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156102a1573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a15056fea165627a7a723058202e512a1df8dbf7513c14646ff33b94f8e9ee902911138575219f157ffdc92ddc0029";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from:"0xf84B27A6D281Df793C67Dc12A2D69FF003BB5732",gas:1000000}, function(err, transactionHash) {
        if (!err) {
            console.log("transactionHash: ", transactionHash);
        }
    })
    .then(console.log);

Overwriting src/BankV2Deploy.js


In [ ]:
_contract.deploy({data:"0x"+_bin}).send({from:"0xf84B27A6D281Df793C67Dc12A2D69FF003BB5732",gas:1000000}, function(err, transactionHash) {
        if (!err) {
            console.log("transactionHash: ", transactionHash);
        }
    }).then(console.log);

In [ ]:
!node src/BankV2Deploy.js

transactionHash:  0xbc4fcaf78d8c8774fb08912b3def8e3365edbd86c53bdf54a2772a270a1b317f
Contract {
  currentProvider: [Getter/Setter],
  _requestManager: RequestManager {
    provider: WebsocketProvider {
      responseCallbacks: {},
      notificationCallbacks: [Array],
      _customTimeout: undefined,
      connection: [W3CWebSocket],
      connected: [Getter],
      lastChunk: null
    },
    providers: {
      WebsocketProvider: [Function: WebsocketProvider],
      HttpProvider: [Function: HttpProvider],
      IpcProvider: [Function: IpcProvider]
    },
    subscriptions: {}
  },
  givenProvider: null,
  providers: {
    WebsocketProvider: [Function: WebsocketProvider],
    HttpProvider: [Function: HttpProvider],
    IpcProvider: [Function: IpcProvider]
  },
  _provider: WebsocketProvider {
    responseCallbacks: {},
    notificationCallbacks: [
      [Function: requestManagerNotification],
      [Function: requestManagerNotification],
      [Function: requestManagerNotification],
   

### 단계 4: 사용

인자의 데이터타잎은 ```address```로 선언되어 있다. ```address``` 즉 계정의 주소를 **따옴표 없이 사용**하면 값이 변경되는 것을 보여주고 있다. args출력을 비교해 보면 원래 함수 인자에서 주어진 값과 다르게 변경된 것을 알 수 있다.

```python
args: 
   { from: '0x21c704354d07f804bab01894e8b4eb4e0eba7451',
     to: '0x778ea91cb0d08927fa4bf3f90a7ccbb700000000',
          // 원래 함수 인자와 다름 0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480
     amount: { [String: '555'] s: 1, e: 2, c: [Object] } }
```

In [16]:
%%writefile src/BankV2Use__.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0xeed4d219cc42d511d17e65fbd3d1b880f62132a8");
var filter = bank.Sent(function (error, result) {
  if (!error)
    console.log(result);
});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, {from:web3.eth.accounts[0],gas:100000, value:555}));

Writing src/BankV2Use.js


In [ ]:
타계정으로 송금을 하고난 후에는 잔고의 변화가 있는지 살펴보자.
```python
> eth.getBalance(eth.accounts[0]);
99988652479999996667
> eth.getBalance(eth.accounts[1]);
100000000000000000000
> eth.getBalance(eth.accounts[0]); //송금 계정의 555 감소
99982721379999996112
> eth.getBalance(eth.accounts[1]); //수신 계정이 555 증가
100000000000000000555
```

타계정은 따옴표로 문자열로 

HttpProvider는 event를 지원하지 않는다. WebsocketProvider

컨트랙에 대해 발생한 과거 이벤트를 모두 ```getPastEvents()``` 함수로 알아보자. 
이벤트가 발생한 블록을 설정할 수 있다. 이 때 범위를 넓게 하면 로그를 찾다가 부하가 발생할 수 있으니 (예를 들어, main net에 최초블록부터 찾으라고 한다면 당연히 부하가 발생할 것이다), 실제 필요한 범위로 좁혀서 설정한다.

```python
bank.getPastEvents('AllEvents', {
    fromBlock: 0,
    toBlock: 'latest'
    }, function(error, result) {
        if (!error) {
            console.log(result);
        } else {
            console.log(error);
        }
    });
```

```python
bank.getPastEvents('Sent', {
    fromBlock: 0,
    toBlock: 'latest'
    }, function(error, result) {
        if (!error) {
            console.log(result);
        } else {
            console.log(error);
        }
    });
```

반환은 다음과 같기 때문에, ```result.returnValues.amount```와 같이 인자명에 키를 더해서 그 값을 볼 수 있다.


```python
 {
    logIndex: 0,
    transactionIndex: 0,
    transactionHash: '0xfaac636e8c059c2fa2a4e6378c33c04a4884f73c4a5041a66809a2d90f4feec8',
    blockHash: '0x81a5162417a5c1d273764c28340ca77637d97088c79d930f5bdf26fb4d21fbf3',
    blockNumber: 23,
    address: '0xfBe8337DFDcBF32247b392FA1F13326A09FedCC6',
    type: 'mined',
    id: 'log_8240e959',
    returnValues: Result {
      '0': '0xf84B27A6D281Df793C67Dc12A2D69FF003BB5732',
      '1': '0x27F9cb3E30ffa2576d2749aa609FC465f6c00cEE',
      '2': '555',
      from: '0xf84B27A6D281Df793C67Dc12A2D69FF003BB5732',
      to: '0x27F9cb3E30ffa2576d2749aa609FC465f6c00cEE',
      amount: '555'
    },
    event: 'Sent',
    signature: '0x3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345',
    raw: {
      data: '0x000000000000000000000000f84b27a6d281df793c67dc12a2d69ff003bb573200000000000000000000000027f9cb3e30ffa2576d2749aa609fc465f6c00cee000000000000000000000000000000000000000000000000000000000000022b',
      topics: [Array]
    }
  }
```

In [1]:
%%writefile src/BankV2Use.js
var Web3=require('web3');
//var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider('http://127.0.0.1:8345'));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var bank = new web3.eth.Contract(_abiArray, '0xfBe8337DFDcBF32247b392FA1F13326A09FedCC6');

bank.events.Sent({
        fromBlock: 'latest',
        toBlock: 'latest'
    }, function(error, result) {
        if (!error) {
            console.log(">> block num: "+result.blockNumber + " to: " + result.returnValues.to + " amount: " + result.returnValues.amount);
        } else {
            console.log(error);
        }
    })
    .on('data', function(event){
        console.log("> data"+event);
    })
    .on('changed', function(event){
        console.log("> changed"+event);
    })
    .on('error', console.error);


bank.methods.sendTo("0x27F9cb3E30ffa2576d2749aa609FC465f6c00cEE").send({from:"0xf84b27a6d281df793c67dc12a2d69ff003bb5732",gas:100000,value:555});


Overwriting src/BankV2Use.js


In [ ]:
!node src/BankV2Use.js

>> block num: 33 to: 0x27F9cb3E30ffa2576d2749aa609FC465f6c00cEE amount: 555
>> block num: 34 to: 0x27F9cb3E30ffa2576d2749aa609FC465f6c00cEE amount: 555


In [ ]:
bank.methods.sendTo("0x27F9cb3E30ffa2576d2749aa609FC465f6c00cEE").send({from:"0xf84b27a6d281df793c67dc12a2d69ff003bb5732",gas:100000,value:555}).on('data', function(event) { console.log(event); } ).on('changed', function(event) { console.log("---"+event); })

In [ ]:
!node src/BankV2Use.js

0x6a693f3558f73c8472a84743a5f419f8d9506716ca2aa2a008d7259ef94079c4
{ address: '0xeed4d219cc42d511d17e65fbd3d1b880f62132a8',
  blockNumber: 263,
  transactionHash: '0x6a693f3558f73c8472a84743a5f419f8d9506716ca2aa2a008d7259ef94079c4',
  transactionIndex: 0,
  blockHash: '0xc83c822d9c9f259aaa746a4f1b871c03352bbf69e649a75fb8591ad73745866a',
  logIndex: 0,
  removed: false,
  event: 'Sent',
  args: 
   { from: '0x21c704354d07f804bab01894e8b4eb4e0eba7451',
     to: '0x778ea91cb0d08927fa4bf3f90a7ccbb700000000',
     amount: { [String: '555'] s: 1, e: 2, c: [Object] } } }


In [ ]:
%%writefile src/BankV2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0xeed4d219cc42d511d17e65fbd3d1b880f62132a8");
var filter = bank.Sent(function (error, result) {
  if (!error)
    console.log(result);
});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.sendTo("0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480", {from:web3.eth.accounts[0],gas:100000, value:555}));

In [ ]:
!node src/BankV2Use.js

0xf7d137adac6cae468b71e9b06741cfec8371f557ef46ce9645dd5c083012237e
{ address: '0xeed4d219cc42d511d17e65fbd3d1b880f62132a8',
  blockNumber: 45482,
  transactionHash: '0xf7d137adac6cae468b71e9b06741cfec8371f557ef46ce9645dd5c083012237e',
  transactionIndex: 0,
  blockHash: '0x407ba95c4153a62c358dbed72fae83de025b8bd1071a51e20551e7f767824b43',
  logIndex: 0,
  removed: false,
  event: 'Sent',
  args: 
   { from: '0x21c704354d07f804bab01894e8b4eb4e0eba7451',
     to: '0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480',
     amount: { [String: '555'] s: 1, e: 2, c: [Object] } } }

받는 주소를 account[1]로 했더니 올바르게 
```python
> eth.getBalance(eth.accounts[1]);
0   <--- 이전 잔고
> eth.pendingTransactions
[{
    blockHash: null,
    blockNumber: null,
    from: "0x21c704354d07f804bab01894e8b4eb4e0eba7451",
    gas: 100000,
    gasPrice: 500000000000,
    hash: "0xa4c9e98767fbe60a0722b3d31647ca351d56ef7a960f00d20edbc6e26a98c94f",
    input: "0x3e58c58c000000000000000000000000778ea91cb0d0879c22ca20c5aea6fbf8cbeed480",
    nonce: 32,
    r: "0x733d32c710bc4d4944da811daa0d639b3a0241cd3994039600c73d69f247a6d",
    s: "0x78476837a95b30c7991af36c32eccbe4ee4f375349adb012db3c8f0c919a0213",
    to: "0x53fd2f60c463f19c8c7bcdb5e699cac94b8300b2",
    transactionIndex: 0,
    v: "0x66",
    value: 555 <--- 전송될 금액
}]
> miner.start(1);admin.sleepBlocks(1);miner.stop();
null
> eth.getBalance(eth.accounts[1]);
555 <--- 이후 잔고
```

In [ ]:
%%writefile src/BankV2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"send","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var sendTest = _contract.at("0x53fd2f60c463f19c8c7bcdb5e699cac94b8300b2");
var filter = sendTest.Sent(function (error, result) {
  if (!error)
    console.log(result);
});
//console.log(sendTest.send(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
//console.log(sendTest.send(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, {from:web3.eth.accounts[0],gas:100000, value:555}));
console.log(sendTest.send(web3.eth.accounts[1], {from:web3.eth.accounts[0],gas:100000, value:555}));

In [1]:
!node src/BankV2Use.js

Overwriting src/SendTest.sol
